# First level GLM - Veridical versus Scrambled

# Imports

In [1]:
import os
import csv
import json
import numpy as np
import pandas as pd
import nibabel as nib
import nilearn as nl
import datetime
from nilearn.plotting import view_img
import numpy.ma as ma

from nilearn import image
from nilearn.image import new_img_like

from nilearn import image
from nilearn import plotting
from nilearn.interfaces import fmriprep
from nilearn.image import load_img
from nilearn.signal import clean
from nilearn.image import resample_img
from nilearn.signal import clean
from nilearn.masking import compute_brain_mask

import nilearn.plotting as plotting
import matplotlib.pyplot as plt
from nilearn.plotting import plot_glass_brain

from nilearn import glm
import nilearn.image as nimg
import nilearn.masking as masking
from nilearn import plotting

# from helper_functions import *

## Notes

This notebook analyzes the data from the 22 participants in our study (23 collected, one excluded because of a head coil issue during scanning). 21 subjects' data was used in full and one subject had one run excluded, with 21 naive subjects and one with knowledge of the study. There were also a few subjects whose data was processed slightly differently due to differences in the scan sequence. For example, for two subjects, they initially saw a repeat experimental run in the run sequence, then completed the missing run afterwards (e.g. they might have seen runs 1, 2, 3, 4, 4, 6, then completed run 5 afterwards). These two subjects are handled in separate chunks of code at the bottom of this notebook.  

# Data paths

In [2]:
# GET SUBJECT DIRECTORIES

data_dir    = '../../data/bids/derivatives/fmriprep/'
events_dir  = '../../data/behavioral/event_timing'
results_dir = 'first_level_GLM'

sub_dirs    = [x for x in os.listdir(data_dir) if 'sub' in x and '.html' not in x 
               and 'sub-112' not in x and 'sub-212' not in x
               and 'sub-101' not in x and 'sub-201' not in x
               and 'sub-017' not in x
              ]

Confirm we have our 23 subjects minus 1 excluded and two unusual (total=20)

In [3]:
len(sub_dirs) 

20

### load mask

In [4]:
mask = nib.load('binary_shaef.nii')
events_path = '../../data/behavioral/event_timing'

# Create GLM 

In [5]:
# create the first_level GLM model
model = nl.glm.first_level.FirstLevelModel(
                                          t_r=2,
                                          signal_scaling=False,
                                          hrf_model='glover',
                                          drift_order=None,
                                          mask_img=mask,
                                          # drift_model='cosine',
                                          drift_model=None,
                                          # Isaac set to none
                                          smoothing_fwhm=4,
                                          standardize=True,
                                          minimize_memory=False,
                                          high_pass= .01 # default value is .01
                                          )

# _ buttons.tsv files

In [6]:
# update timing labels for 1 & 2 timing files

ej_files      = [ x for x in os.listdir(events_dir) if '_buttons.tsv' in x ] # 'events_judgement' in x ]
ej_files_full = [ events_path + f for f in ej_files]


### Prep the events files and contrasts for different GLM comparisons

In [7]:
events_list   = ['labeled-type-videos']

events_files  = [ x + '.tsv' for x in events_list ] 

contrast_list = ['veridical - scrambled']

### load runs to exclude 

In [8]:
pd.read_csv('../../data/behavioral/exclude_runs_behavioral.csv')

,Unnamed: 0,run,moviestim,subject
0,2,3.0,15,new MRI Behavioral Data/subject_21


# Functions

In [9]:
def get_data(image, mask=False, avg_mask=np.nan, full_output=False):
    '''
    inputs  : mask_type - string - mask_type (options: gm (grey matter), 
                                                    whole-brain, 
                                                    wm (white matter))
              image     - string - path to mri data for a single run
    outputs : masked_data - ???? - brain data with grey matter mask applied
    '''
    
    # Load your fMRI data
    fmri_image = nib.load(image)
    
    if mask == True:
        
        print("you're already passing mask into the model!")
    
    else:
        
        final_data = fmri_image.get_fdata()
    
    if full_output:
        
        affine = fmri_image.affine
        header = fmri_image.header
        shape  = final_data.shape
        
        return(fmri_image, final_data, affine, header, shape)
    
    else:
        
        return(final_data)

In [10]:
def get_good_runs(data_dir, subject, percent_frames=.30, fwd=.5, exclude=[]):
    '''
    input   :  data_dir  - string - path to subject directories
               subject   - string - subject directory
    output  :  good_runs - list   - list of runs with <30% of frames with motion (>=.5 FWD)
    '''
    
    good_runs   = []
    regressors  = []
    
    for r in ['01','02','03','04','05','06']:
        
        if r not in exclude:
        
            df      = pd.read_csv(data_dir+subject+'/ses-01/func/'+subject+'_ses-01_task-PredictingAttention_run-'+r+'_desc-confounds_timeseries.tsv', sep="\t")
            percent = df[df['framewise_displacement']>=fwd].shape[0] / df.shape[0]

            # if bad motion in less than 30% of frames --> good run
            if percent < percent_frames:
                good_runs.append(r)

                l = list(df['framewise_displacement']>.5)
                l_new = [int(x) for x in l]
                regressors.append(l_new)
        
    return(good_runs,regressors)



### Do the GLM

In [2]:
# we are excluding subject 21 run 3

for subject in sub_dirs:  
    
    for events_file, contrast in zip( events_files, contrast_list ):
        
        new_confound_list = []
            
        print(subject); print(contrast)

        if subject == 'sub-021':
            # get runs without excessive motion, exclude run 3
            runs,add_regressor_columns = get_good_runs(data_dir, subject, exclude=['03'])
            print('excluded runs 3 for subject 21. included:')
            print(runs)
            
        else:
            # get runs without excessive motion
            runs,add_regressor_columns = get_good_runs(data_dir, subject)
        
        # select the images for the good runs
        image_list = [ data_dir+subject+'/ses-01/func/'+subject+'_ses-01_task-PredictingAttention_run-'+x+'_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz' for x in runs ]

        # get confounds
        strategy   = [ 'motion','wm_csf','global_signal','compcor','scrub','high_pass' ]
        confounds  = [ nl.interfaces.fmriprep.load_confounds(x, 
                                                            strategy = strategy, 
                                                            fd_threshold=0.5,
                                                            n_compcor = 5,
                                                            motion ='basic',
                                                            demean = False) for x in image_list ]
        confound_list = [x[0] for x in confounds]
        
        new_confound_list = confound_list 
        
        # get sample masks
        sm_list = [x[1] for x in confounds]

        # replace any cases of None with array listing all frames, 0-109
        sm_list = [np.arange(110) if x is None else x for x in sm_list]

        # get data
        full_data = []; affine = []; header = []; shape = []

        for i in image_list:
            the_image,f,a,h,s = get_data(i, mask=False, avg_mask=None, full_output=True) 
            affine.append(a); header.append(h); shape.append(s)            
            full_data.append(the_image)

        cleaned_images = full_data #[nib.Nifti1Image(a, affine=None) for a in full_data]

        # load event timings
        event_timing_files = [events_dir+'/'+subject+'_ses-01_task-PredictingAttention_run-'+r+'_'+events_file for r in runs]

        event_dfs = [ pd.read_table(x) for x in event_timing_files ]

        # make design matrices
        design_matrices = []

        for e,c in zip(event_dfs, confound_list):

            TR_array   = np.array([x*2 for x in np.arange(110)])
            matrix = glm.first_level.make_first_level_design_matrix(TR_array, 
                                                                    e, 
                                                                    hrf_model='glover', 
                                                                    drift_model=None,
                                                                    add_regs = c)
            design_matrices.append(matrix)
            print('ready to fit the model')


        print(cleaned_images[0].shape)
        print(new_confound_list[0].shape)
        print(design_matrices[0].shape)
        print(event_dfs[0].shape)
        print(sm_list[0].shape)

        # fit the model
        fitted_model = model.fit(cleaned_images, 
                                 confounds       = new_confound_list, 
                                 design_matrices = design_matrices,
                                 events          = event_dfs,
                                 sample_masks    = sm_list) 

        print('ready to compute contrasts')
        # plot the result
        z_score  = fitted_model.compute_contrast(contrast, output_type='z_score')
        eff_size = fitted_model.compute_contrast(contrast, output_type='effect_size')
        # NOTE: we'll use effect_size for second-level GLM
        print('contrasts computed')

        # make folder path
        path_base = 'first_level_GLM' 
        directory_path = path_base 

        print('directory path created')

        # make folder for this batch, if it doesn't exist
        if not os.path.exists(directory_path):
            os.makedirs(directory_path)
            print("Directory created:", directory_path)
        else:
            print("Directory already exists:", directory_path)

        # contrast for filename
        contrast_label = contrast.replace(' ','-')

        # timing label
        now   = datetime.datetime.now()
        ctime = now.ctime()
        date_time = ctime.replace(' ','-')

        # save contrasts
        z_score.to_filename(directory_path + '/' + events_file[:-4] + '_' + contrast_label + '_' + date_time + '_' + str(subject) + '_z-score.nii.gz')
        eff_size.to_filename(directory_path + '/' + events_file[:-4] + '_' + contrast_label + '_' + date_time + '_' + str(subject) + '_eff-size.nii.gz')

        print('contrasts saved out')
        print(directory_path + '/' + events_file[:-4] + '_' + contrast_label + '_' + date_time + '_' + str(subject) + '_z-score.nii.gz')


### Now unusual subjects !

#### Subject 1

In [12]:
sub1 = pd.read_csv('../../data/behavioral/subject_1_unusual_run_order.csv')
sub1

,Unnamed: 0,0
0,subject,subject_1
1,behavioral_runs,"['1', '2', '3', '4', '4', '5', '6']"


In [13]:
# subject 101 has runs labeled 01, 02, 03, 04, 05, 06

# they correspond to           1,  2,  3,  4, drop, 5



# subject 201 has runs labeled 01

# they correspond to           6

In [5]:
# let's start loading the data for subject 101

# we can treat everything normally for runs 1-4, so let's exclude the rest for now

for subject in ['sub-101']:  
    
    for events_file, contrast in zip( events_files, contrast_list ):
        
        new_confound_list = []
        
        path_base = 'first_level_GLM' 
            
        print(subject)

        if subject == 'sub-101':
            runs,add_regressor_columns = get_good_runs(data_dir, subject, exclude=['05','06'])
            print('excluded runs 5, 6 for subject 101. included:')
            print(runs)
            
        
        # select the images for the good runs
        image_list = [ data_dir+subject+'/ses-01/func/'+subject+'_ses-01_task-PredictingAttention_run-'+x+'_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz' for x in ['01','02','03','04'] ]
        
        # get confounds
        strategy   = [ 'motion','wm_csf','global_signal','compcor','scrub','high_pass' ]
        confounds  = [ nl.interfaces.fmriprep.load_confounds(x, 
                                                            strategy = strategy, 
                                                            fd_threshold=0.5,
                                                            n_compcor = 5,
                                                            motion ='basic',
                                                            demean = False) for x in image_list ]
        confound_list = [x[0] for x in confounds]
        
        new_confound_list = confound_list 
        
        # commented lines below because we already do headscrubbing with nilearn
        
#         for add,c in zip(add_regressor_columns,confound_list):
#             d = pd.DataFrame({'fwd':add})
#             new_confound_list.append(pd.concat([c,d],axis=1))

        # get sample masks
        sm_list = [x[1] for x in confounds]

        # replace any cases of None with array listing all frames, 0-109
        sm_list = [np.arange(110) if x is None else x for x in sm_list]

        # get data
        full_data = []; affine = []; header = []; shape = []

        for i in image_list:
            the_image,f,a,h,s = get_data(i, mask=False, avg_mask=None, full_output=True) 
            affine.append(a); header.append(h); shape.append(s)            
            full_data.append(the_image)

        cleaned_images = full_data #[nib.Nifti1Image(a, affine=None) for a in full_data]

        # load event timings
        event_timing_files = [events_dir+'/sub-001_ses-01_task-PredictingAttention_run-'+r+'_'+events_file for r in runs]

        event_dfs = [ pd.read_table(x) for x in event_timing_files ]

        # make design matrices
        design_matrices = []

        for e,c in zip(event_dfs, confound_list):

            TR_array   = np.array([x*2 for x in np.arange(110)])
            matrix = glm.first_level.make_first_level_design_matrix(TR_array, 
                                                                    e, 
                                                                    hrf_model='glover', 
                                                                    drift_model=None,
                                                                    add_regs = c)
            design_matrices.append(matrix)
            print('ready to fit the model')


        print(cleaned_images[0].shape)
        print(new_confound_list[0].shape)
        print(design_matrices[0].shape)
        print(event_dfs[0].shape)
        print(sm_list[0].shape)




In [15]:
o_cleaned_images    = cleaned_images
o_new_confound_list = new_confound_list
o_design_matrices   = design_matrices
o_event_dfs         = event_dfs
o_sm_list           = sm_list

print(len(cleaned_images))
print(len(new_confound_list))
print(len(design_matrices))
print(len(event_dfs))
print(len(sm_list))

4
4
4
4
4


In [16]:
# now we'll add the fifth element:
# it will be run 6 of the MRI data
# it will be run 5 of the timing data

for subject in ['sub-101']:  

    
    for events_file, contrast in zip( events_files, contrast_list ):
        
        new_confound_list = []

        if subject == 'sub-101':
            # get runs without excessive motion, exclude runs 2, 3, 4
            runs,add_regressor_columns = get_good_runs(data_dir, subject, exclude=['01','02','03','04','05'])
            print('excluded runs 1, 2, 3, 4, 5 for subject 101. included:')
            print(runs)
        
        # select the images for the good runs
        image_list = [ data_dir+subject+'/ses-01/func/sub-101_ses-01_task-PredictingAttention_run-06_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz' ]
        print(len(image_list))
        
        # get confounds
        strategy   = [ 'motion','wm_csf','global_signal','compcor','scrub','high_pass' ]
        confounds  = [ nl.interfaces.fmriprep.load_confounds(x, 
                                                            strategy = strategy, 
                                                            fd_threshold=0.5,
                                                            n_compcor = 5,
                                                            motion ='basic',
                                                            demean = False) for x in image_list ]

        confound_list = [x[0] for x in confounds]
        
        new_confound_list = confound_list 
        
        # commented lines below because we already do headscrubbing with nilearn
        
#         for add,c in zip(add_regressor_columns,confound_list):
#             d = pd.DataFrame({'fwd':add})
#             new_confound_list.append(pd.concat([c,d],axis=1))

        # get sample masks
        sm_list = [x[1] for x in confounds]

        # replace any cases of None with array listing all frames, 0-109
        sm_list = [np.arange(110) if x is None else x for x in sm_list]

        # get data
        full_data = []; affine = []; header = []; shape = []

        for i in image_list:
            the_image,f,a,h,s = get_data(i, mask=False, avg_mask=None, full_output=True) 
            affine.append(a); header.append(h); shape.append(s)            
            full_data.append(the_image)

        cleaned_images = full_data #[nib.Nifti1Image(a, affine=None) for a in full_data]

        # load event timings
        event_timing_files = [events_dir+'/sub-001_ses-01_task-PredictingAttention_run-05_'+events_file for r in runs]

        event_dfs = [ pd.read_table(x) for x in event_timing_files ]

        # make design matrices
        design_matrices = []

        for e,c in zip(event_dfs, confound_list):

            TR_array   = np.array([x*2 for x in np.arange(110)])
            matrix = glm.first_level.make_first_level_design_matrix(TR_array, 
                                                                    e, 
                                                                    hrf_model='glover', 
                                                                    drift_model=None,
                                                                    add_regs = c)
            design_matrices.append(matrix)
            print('ready to fit the model')


        print(cleaned_images[0].shape)
        print(new_confound_list[0].shape)
        print(design_matrices[0].shape)
        print(event_dfs[0].shape)
        print(sm_list[0].shape)



excluded runs 1, 2, 3, 4, 5 for subject 101. included:
['06']
1
ready to fit the model
(78, 93, 78, 110)
(110, 16)
(110, 19)
(20, 5)
(110,)


/usr/people/kz0108/.local/lib/python3.7/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: response_time, stim_file
  "The following unexpected columns "


In [17]:
print(len(cleaned_images))
print(len(new_confound_list))
print(len(design_matrices))
print(len(event_dfs))
print(len(sm_list))

1
1
1
1
1


In [18]:
print(len(o_cleaned_images))
print(len(o_new_confound_list))
print(len(o_design_matrices))
print(len(o_event_dfs))
print(len(o_sm_list))

4
4
4
4
4


In [19]:
o_cleaned_images.append(cleaned_images[0])
o_new_confound_list.append(new_confound_list[0])
o_design_matrices.append(design_matrices[0])
o_event_dfs.append(event_dfs[0])
o_sm_list.append(sm_list[0])

In [20]:
print(len(o_cleaned_images))
print(len(o_new_confound_list))
print(len(o_design_matrices))
print(len(o_event_dfs))
print(len(o_sm_list))

5
5
5
5
5


In [21]:
# now we'll add the sixth element:
# it will be run 1 of the MRI data for subject 201
# it will be run 6 of the timing data for subject 001

for subject in ['sub-201']:  
    
    for events_file, contrast in zip( events_files, contrast_list ):
        
        new_confound_list = [] 
        
        if subject == 'sub-201':
            # get runs without excessive motion, exclude runs 2, 3, 4
            runs,add_regressor_columns = get_good_runs(data_dir, subject, exclude=['02','03','04','05','06'])
            print('excluded runs 2, 3, 4, 5, 6. included:')
            print(runs)
            
        else:
            # get runs without excessive motion
            runs,add_regressor_columns = get_good_runs(data_dir, subject)
        
        # select the images for the good runs
        image_list = [ data_dir+subject+'/ses-01/func/sub-201_ses-01_task-PredictingAttention_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz' ]
        
        # get confounds
        strategy   = [ 'motion','wm_csf','global_signal','compcor','scrub','high_pass' ]
        confounds  = [ nl.interfaces.fmriprep.load_confounds(x, 
                                                            strategy = strategy, 
                                                            fd_threshold=0.5,
                                                            n_compcor = 5,
                                                            motion ='basic',
                                                            demean = False) for x in image_list ]
        confound_list = [x[0] for x in confounds]
        
        new_confound_list = confound_list 
        
        # commented lines below because we already do headscrubbing with nilearn
        
#         for add,c in zip(add_regressor_columns,confound_list):
#             d = pd.DataFrame({'fwd':add})
#             new_confound_list.append(pd.concat([c,d],axis=1))

        # get sample masks
        sm_list = [x[1] for x in confounds]

        # replace any cases of None with array listing all frames, 0-109
        sm_list = [np.arange(110) if x is None else x for x in sm_list]

        # get data
        full_data = []; affine = []; header = []; shape = []

        for i in image_list:
            the_image,f,a,h,s = get_data(i, mask=False, avg_mask=None, full_output=True) 
            affine.append(a); header.append(h); shape.append(s)            
            full_data.append(the_image)

        cleaned_images = full_data #[nib.Nifti1Image(a, affine=None) for a in full_data]

        # load event timings
        event_timing_files = [events_dir+'/sub-001_ses-01_task-PredictingAttention_run-06_'+events_file for r in runs]

        event_dfs = [ pd.read_table(x) for x in event_timing_files ]

        # make design matrices
        design_matrices = []

        for e,c in zip(event_dfs, confound_list):

            TR_array   = np.array([x*2 for x in np.arange(110)])
            matrix = glm.first_level.make_first_level_design_matrix(TR_array, 
                                                                    e, 
                                                                    hrf_model='glover', 
                                                                    drift_model=None,
                                                                    add_regs = c)
            design_matrices.append(matrix)
            print('ready to fit the model')


        print(cleaned_images[0].shape)
        print(new_confound_list[0].shape)
        print(design_matrices[0].shape)
        print(event_dfs[0].shape)
        print(sm_list[0].shape)



excluded runs 2, 3, 4, 5, 6 for subject 12. included:
['01']
ready to fit the model
(78, 93, 78, 110)
(110, 16)
(110, 19)
(20, 5)
(108,)


/usr/people/kz0108/.local/lib/python3.7/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: response_time, stim_file
  "The following unexpected columns "


In [22]:
print(len(cleaned_images))
print(len(new_confound_list))
print(len(design_matrices))
print(len(event_dfs))
print(len(sm_list))

1
1
1
1
1


In [23]:
print(len(o_cleaned_images))
print(len(o_new_confound_list))
print(len(o_design_matrices))
print(len(o_event_dfs))
print(len(o_sm_list))

5
5
5
5
5


In [24]:
o_cleaned_images.append(cleaned_images[0])
o_new_confound_list.append(new_confound_list[0])
o_design_matrices.append(design_matrices[0])
o_event_dfs.append(event_dfs[0])
o_sm_list.append(sm_list[0])

In [3]:
# fit the model
fitted_model = model.fit(o_cleaned_images, 
                         confounds       = o_new_confound_list, 
                         design_matrices = o_design_matrices,
                         events          = o_event_dfs,
                         sample_masks    = o_sm_list) 

print('ready to compute contrasts')
# plot the result
z_score  = fitted_model.compute_contrast(contrast, output_type='z_score')
eff_size = fitted_model.compute_contrast(contrast, output_type='effect_size')
# NOTE: we'll use z_score for visualizing and effect_size for second-level GLM
print('contrasts computed')

# make folder path
path_base = 'first_level_GLM' 
directory_path = path_base 

print('directory path created')

# make folder for this batch, if it doesn't exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print("Directory created:", directory_path)
else:
    print("Directory already exists:", directory_path)

# contrast for filename
contrast_label = contrast.replace(' ','-')

# timing label
now   = datetime.datetime.now()
ctime = now.ctime()
date_time = ctime.replace(' ','-')

# save contrasts
z_score.to_filename(directory_path + '/' + events_file[:-4] + '_' + contrast_label + '_' + date_time + '_' + 'sub-001' + '_z-score.nii.gz')
eff_size.to_filename(directory_path + '/' + events_file[:-4] + '_' + contrast_label + '_' + date_time + '_' + 'sub-001' + '_eff-size.nii.gz')

print('contrasts saved out')
print(directory_path + '/' + contrast_label + '_' + date_time + '_' + 'sub-001' + '_z-score.nii.gz')


### Now, subject 12

In [26]:
sub1 = pd.read_csv('../../data/behavioral/subject_12_unusual_run_order.csv')
sub1

,Unnamed: 0,1
0,subject,subject_12
1,behavioral_runs,"['1', '2', '3', '3', '5', '6', '4']"


In [27]:
# subject 212 has runs labeled 01, 02, 03, 04, 05, 06

# they correspond to           1,  2,  3,  drop, 5, 6



# subject 201 has runs labeled 01

# they correspond to           4

In [28]:
# let's start loading the data for subject 212

for subject in ['sub-212']:  
    
    for events_file, contrast in zip( events_files, contrast_list ):
        
        new_confound_list = []
        
        
        ######## THIS CODE IS TO GET NEW SUBJECTS WHO WERE MISSING BEFORE ########
        
        # only uncomment if incorporating new subjects into existing first-level analysis
        
        # make folder path
        path_base = 'first_level_GLM' 
        directory_path = path_base 
        
        contrast_label = contrast.replace(' ','-')
        #check_file     = [ x for x in os.listdir(directory_path) if str(subject) in x and contrast_label in x and '_eff-size.nii.gz' in x ]
        
        ###########################################################################
            
        print(subject); print(contrast_label)

        if subject == 'sub-212':
            runs,add_regressor_columns = get_good_runs(data_dir, subject, exclude=['04'])
            print('excluded run 4 for subject 12. included:')
            print(runs)
            
        else:
            # get runs without excessive motion
            runs,add_regressor_columns = get_good_runs(data_dir, subject)
        
        # select the images for the good runs
        image_list = [ data_dir+subject+'/ses-01/func/'+subject+'_ses-01_task-PredictingAttention_run-'+x+'_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz' for x in ['01','02','03','05','06'] ]
        
        # get confounds
        strategy   = [ 'motion','wm_csf','global_signal','compcor','scrub','high_pass' ]
        confounds  = [ nl.interfaces.fmriprep.load_confounds(x, 
                                                            strategy = strategy, 
                                                            fd_threshold=0.5,
                                                            n_compcor = 5,
                                                            motion ='basic',
                                                            demean = False) for x in image_list ]
        confound_list = [x[0] for x in confounds]
        
        new_confound_list = confound_list 
        
        # commented lines below because we already do headscrubbing with nilearn
        
#         for add,c in zip(add_regressor_columns,confound_list):
#             d = pd.DataFrame({'fwd':add})
#             new_confound_list.append(pd.concat([c,d],axis=1))

        # get sample masks
        sm_list = [x[1] for x in confounds]

        # replace any cases of None with array listing all frames, 0-109
        sm_list = [np.arange(110) if x is None else x for x in sm_list]

        # get data
        full_data = []; affine = []; header = []; shape = []

        for i in image_list:
            the_image,f,a,h,s = get_data(i, mask=False, avg_mask=None, full_output=True) 
            affine.append(a); header.append(h); shape.append(s)            
            full_data.append(the_image)

        cleaned_images = full_data #[nib.Nifti1Image(a, affine=None) for a in full_data]

        # load event timings
        event_timing_files = [events_dir+'/sub-012_ses-01_task-PredictingAttention_run-'+r+'_'+events_file for r in runs]

        event_dfs = [ pd.read_table(x) for x in event_timing_files ]

        # make design matrices
        design_matrices = []

        for e,c in zip(event_dfs, confound_list):

            TR_array   = np.array([x*2 for x in np.arange(110)])
            matrix = glm.first_level.make_first_level_design_matrix(TR_array, 
                                                                    e, 
                                                                    hrf_model='glover', 
                                                                    drift_model=None,
                                                                    add_regs = c)
            design_matrices.append(matrix)
            print('ready to fit the model')


        print(cleaned_images[0].shape)
        print(new_confound_list[0].shape)
        print(design_matrices[0].shape)
        print(event_dfs[0].shape)
        print(sm_list[0].shape)




sub-212
veridical---scrambled
excluded run 4 for subject 12. included:
['01', '02', '03', '05', '06']
ready to fit the model
ready to fit the model
ready to fit the model
ready to fit the model
ready to fit the model
(78, 93, 78, 110)
(110, 16)
(110, 19)
(20, 5)
(110,)


/usr/people/kz0108/.local/lib/python3.7/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: response_time, stim_file
  "The following unexpected columns "
/usr/people/kz0108/.local/lib/python3.7/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: response_time, stim_file
  "The following unexpected columns "
/usr/people/kz0108/.local/lib/python3.7/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: response_time, stim_file
  "The following unexpected columns "
/usr/people/kz0108/.local/lib/python3.7/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: response_time, stim_file
  "The following unexpected columns "
/usr

In [29]:
o_cleaned_images    = cleaned_images
o_new_confound_list = new_confound_list
o_design_matrices   = design_matrices
o_event_dfs         = event_dfs
o_sm_list           = sm_list

print(len(cleaned_images))
print(len(new_confound_list))
print(len(design_matrices))
print(len(event_dfs))
print(len(sm_list))

5
5
5
5
5


In [30]:
# now we'll add the sixth element:
# it will be run 1 of the MRI data for subject 112
# it will be run 4 of the timing data for subject 12

for subject in ['sub-112']:  
    
    for events_file, contrast in zip( events_files, contrast_list ):
        
        new_confound_list = [] 
        
        if subject == 'sub-112':
            # get runs without excessive motion, exclude runs 2, 3, 4
            runs,add_regressor_columns = get_good_runs(data_dir, subject, exclude=['02','03','04','05','06'])
            print('excluded runs 2, 3, 4, 5, 6 for subject 12. included:')
            print(runs)
        
        # select the images for the good runs
        image_list = [ data_dir+subject+'/ses-01/func/sub-112_ses-01_task-PredictingAttention_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz' ]
        
        # get confounds
        strategy   = [ 'motion','wm_csf','global_signal','compcor','scrub','high_pass' ]
        confounds  = [ nl.interfaces.fmriprep.load_confounds(x, 
                                                            strategy = strategy, 
                                                            fd_threshold=0.5,
                                                            n_compcor = 5,
                                                            motion ='basic',
                                                            demean = False) for x in image_list ]
        confound_list = [x[0] for x in confounds]
        
        new_confound_list = confound_list 
        
        # commented lines below because we already do headscrubbing with nilearn
        
#         for add,c in zip(add_regressor_columns,confound_list):
#             d = pd.DataFrame({'fwd':add})
#             new_confound_list.append(pd.concat([c,d],axis=1))

        # get sample masks
        sm_list = [x[1] for x in confounds]

        # replace any cases of None with array listing all frames, 0-109
        sm_list = [np.arange(110) if x is None else x for x in sm_list]

        # get data
        full_data = []; affine = []; header = []; shape = []

        for i in image_list:
            the_image,f,a,h,s = get_data(i, mask=False, avg_mask=None, full_output=True) 
            affine.append(a); header.append(h); shape.append(s)            
            full_data.append(the_image)

        cleaned_images = full_data #[nib.Nifti1Image(a, affine=None) for a in full_data]

        # load event timings
        event_timing_files = [events_dir+'/sub-012_ses-01_task-PredictingAttention_run-04_'+events_file for r in runs]

        event_dfs = [ pd.read_table(x) for x in event_timing_files ]

        # make design matrices
        design_matrices = []

        for e,c in zip(event_dfs, confound_list):

            TR_array   = np.array([x*2 for x in np.arange(110)])
            matrix = glm.first_level.make_first_level_design_matrix(TR_array, 
                                                                    e, 
                                                                    hrf_model='glover', 
                                                                    drift_model=None,
                                                                    add_regs = c)
            design_matrices.append(matrix)
            print('ready to fit the model')


        print(cleaned_images[0].shape)
        print(new_confound_list[0].shape)
        print(design_matrices[0].shape)
        print(event_dfs[0].shape)
        print(sm_list[0].shape)


excluded runs 2, 3, 4, 5, 6 for subject 12. included:
['01']
ready to fit the model
(78, 93, 78, 110)
(110, 16)
(110, 19)
(20, 5)
(108,)


/usr/people/kz0108/.local/lib/python3.7/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: response_time, stim_file
  "The following unexpected columns "


In [31]:
print(len(cleaned_images))
print(len(new_confound_list))
print(len(design_matrices))
print(len(event_dfs))
print(len(sm_list))

1
1
1
1
1


In [32]:
print(len(o_cleaned_images))
print(len(o_new_confound_list))
print(len(o_design_matrices))
print(len(o_event_dfs))
print(len(o_sm_list))

5
5
5
5
5


In [33]:
o_cleaned_images.append(cleaned_images[0])
o_new_confound_list.append(new_confound_list[0])
o_design_matrices.append(design_matrices[0])
o_event_dfs.append(event_dfs[0])
o_sm_list.append(sm_list[0])

In [34]:
print(len(o_cleaned_images))
print(len(o_new_confound_list))
print(len(o_design_matrices))
print(len(o_event_dfs))
print(len(o_sm_list))

6
6
6
6
6


In [4]:
# fit the model
fitted_model = model.fit(o_cleaned_images, 
                         confounds       = o_new_confound_list, 
                         design_matrices = o_design_matrices,
                         events          = o_event_dfs,
                         sample_masks    = o_sm_list) 

print('ready to compute contrasts')
# plot the result
z_score  = fitted_model.compute_contrast(contrast, output_type='z_score')
eff_size = fitted_model.compute_contrast(contrast, output_type='effect_size')
# NOTE: we'll use z_score for visualizing and effect_size for second-level GLM
print('contrasts computed')

# make folder path
path_base = 'first_level_GLM' 
directory_path = path_base 

print('directory path created')

# make folder for this batch, if it doesn't exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print("Directory created:", directory_path)
else:
    print("Directory already exists:", directory_path)

# contrast for filename
contrast_label = contrast.replace(' ','-')

# timing label
now   = datetime.datetime.now()
ctime = now.ctime()
date_time = ctime.replace(' ','-')

# save contrasts
z_score.to_filename(directory_path + '/' + events_file[:-4] + '_' + contrast_label + '_' + date_time + '_' + 'sub-012' + '_z-score.nii.gz')
eff_size.to_filename(directory_path + '/' + events_file[:-4] + '_' + contrast_label + '_' + date_time + '_' + 'sub-012' + '_eff-size.nii.gz')

print('contrasts saved out')
print(directory_path + '/' + contrast_label + '_' + date_time + '_' + 'sub-012' + '_z-score.nii.gz')
